In [1]:
import torch
import torchvision
from torchvision import models, transforms
from matplotlib import pyplot as plt
import numpy as np
import os
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.io import read_image
from torchvision.transforms import ToTensor
from PIL import Image

>>>>>>>>>>>INTERACT HERE

In [20]:
path_to_model = '/Users/david/Desktop/shelter_dogs_research/Trials/trial1_pup1/Trial1_docs/trial1.0_model.pt'
path_to_folder = '/Users/david/Desktop/shelter_dogs_research/Trials/trial1_pup1/datasets/inpainted/test'

>>>>>>>>>>>>>>>>>>RUN ALL

In [7]:
class_names = os.listdir(path_to_folder)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.jit.load(path_to_model)
model.eval()

RecursiveScriptModule(
  original_name=ResNet
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
  (relu): RecursiveScriptModule(original_name=ReLU)
  (maxpool): RecursiveScriptModule(original_name=MaxPool2d)
  (layer1): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_name=Conv2d)
      (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
    )
    (1): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_name=Conv2d)

In [8]:
data_transforms = {
    'test': transforms.Compose([
        #transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.Grayscale(),
        transforms.Grayscale(3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}


In [9]:
def imshow(inp, title=None):
    """Display image for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [10]:
def visualize(model, img_path):
    img = Image.open(img_path)
    img = img.convert('RGB')
    img = data_transforms['test'](img)
    img = img.unsqueeze(0)
    img = img.to(device)

    with torch.no_grad():
        outputs = model(img)
        _, preds = torch.max(outputs, 1)

       # ax = plt.subplot(2,2,1)
       # ax.axis('off')
       # ax.set_title(f'Predicted: {class_names[preds[0]]}')
       # imshow(img.cpu().data[0])
    return class_names[preds[0]]

In [11]:
def confusion_matrix(model, folder_path):
    catagories = os.listdir(folder_path)
    if '.DS_Store' in catagories:
        catagories.remove('.DS_Store')
    num_catagories = len(catagories)
    #import confusion matrix
    value_library = {}
    confusion_matrix = np.zeros((num_catagories, num_catagories))
    catagory_list = os.listdir(folder_path)
    if '.DS_Store' in catagory_list:
        catagory_list.remove('.DS_Store')
    for catagory in catagory_list:
        catagory_path = os.path.join(folder_path, catagory)
        img_list = os.listdir(catagory_path)
        if '.DS_Store' in img_list:
            img_list.remove('.DS_Store')
        
        #shuffle order of img_list
        np.random.shuffle(img_list)
        for img in img_list:
            img_path = os.path.join(catagory_path, img)
            predicted = visualize(model, img_path)
            predicted_index = catagory_list.index(predicted)
            actual_index = catagory_list.index(catagory)
            if predicted_index != actual_index:
                visualize_wimg(model,img_path)
                print('actual identity:' , catagory)
            confusion_matrix[actual_index][predicted_index] += 1
            #write functions to calculate true positive, true negative, false positive, false negative
    for catagory in catagory_list:
        catagory_index = catagory_list.index(catagory)
        true_positive = confusion_matrix[catagory_index][catagory_index]
        false_negative = confusion_matrix[catagory_index][:].sum() - true_positive
        num = 0
        for row in confusion_matrix:
            num = num + row[catagory_index]
        false_positive = num - true_positive
        
        true_negative = confusion_matrix[:][:].sum() - confusion_matrix[catagory_index][:].sum() - confusion_matrix[:][catagory_index].sum() + confusion_matrix[catagory_index][catagory_index]
        values = {}
        values['true_positive'] = true_positive
        values['true_negative'] = true_negative
        values['false_positive'] = false_positive
        values['false_negative'] = false_negative
        value_library[catagory] = values
        
    return value_library



In [12]:
def confusion_matrix_2cat(model, folder_path):
    num_catagories = len(os.listdir(folder_path))
    value_library = {}
    confusion_matrix = np.zeros((num_catagories, num_catagories))
    catagory_list = os.listdir(folder_path)
    for catagory in catagory_list:
        catagory_path = os.path.join(folder_path, catagory)
        img_list = os.listdir(catagory_path)
        for img in img_list:
            img_path = os.path.join(catagory_path, img)
            predicted = visualize(model, img_path)
            predicted_index = catagory_list.index(predicted)
            actual_index = catagory_list.index(catagory)
            
            confusion_matrix[actual_index][predicted_index] += 1
            #write functions to calculate true positive, true negative, false positive, false negative
    for catagory in catagory_list:
        catagory_index = catagory_list.index(catagory)
        true_positive = confusion_matrix[catagory_index][catagory_index]
        if catagory_index == 0:
            false_negative = confusion_matrix[catagory_index][catagory_index+1]
        if catagory_index == 1:
            false_negative = confusion_matrix[catagory_index][catagory_index-1]
        false_positive = confusion_matrix[:][catagory_index].sum() - true_positive
        true_negative = confusion_matrix[:][:].sum() - confusion_matrix[catagory_index][:].sum() - confusion_matrix[:][catagory_index].sum() + confusion_matrix[catagory_index][catagory_index]
        values = {}
        values['true_positive'] = true_positive
        values['true_negative'] = true_negative
        values['false_positive'] = false_positive
        values['false_negative'] = false_negative
        value_library[catagory] = values
    return value_library
 

In [13]:
def evaluation_metrics(value_library):
    evaluation_metrics = {}
    for catagory in value_library:
        values = value_library[catagory]
        true_positive = values['true_positive']
        true_negative = values['true_negative']
        false_positive = values['false_positive']
        false_negative = values['false_negative']
        accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)
        precision = true_positive / (true_positive + false_positive)
        recall = true_positive / (true_positive + false_negative)
        f1_score = 2 * ((precision * recall) / (precision + recall))
        evaluation_metrics[catagory] = {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1_score': f1_score}
    return evaluation_metrics


In [14]:
def average_evaluation_metrics(evaluation_metrics):
    average_evaluation_metrics = {}
    accuracy = 0
    precision = 0
    recall = 0
    f1_score = 0
    for catagory in evaluation_metrics:
        values = evaluation_metrics[catagory]
        accuracy += values['accuracy']
        precision += values['precision']
        recall += values['recall']
        f1_score += values['f1_score']
    average_evaluation_metrics['accuracy'] = accuracy / len(evaluation_metrics)
    average_evaluation_metrics['precision'] = precision / len(evaluation_metrics)
    average_evaluation_metrics['recall'] = recall / len(evaluation_metrics)
    average_evaluation_metrics['f1_score'] = f1_score / len(evaluation_metrics)
    return average_evaluation_metrics


In [15]:
def visualize_eval(value_library, evaluation_metric_lib, average_evaluation_metrics_lib):
    print('\n' +'\n' +'~~~~~~~~~~~~~~~~~~~~~~~EVALUATION~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    for value in value_library:
        print(value + ':')
        print(value_library[value])
        print('catagory metrics: ')
        print(evaluation_metric_lib[value])
        print ('\n')
    print('########################################')
    print('average evaluation metrics: ')
    for val in average_evaluation_metrics_lib:
        print(val + ': ' + str(average_evaluation_metrics_lib[val]))
    print('########################################')

In [16]:
def evaluate(model, folder_path, cat_num):
    if cat_num == 2:
        value_library = confusion_matrix_2cat(model, folder_path)
    else:
        value_library = confusion_matrix(model, folder_path)
    evaluation_metric_lib = evaluation_metrics(value_library)
    average_evaluation_metrics_lib = average_evaluation_metrics(evaluation_metric_lib)
    visualize_eval(value_library, evaluation_metric_lib, average_evaluation_metrics_lib)
    return average_evaluation_metrics_lib

>>>>>>>>>>>>>INTERACT HERE

In [21]:
class_names = os.listdir(path_to_folder)
if '.DS_Store' in class_names:
    class_names.remove('.DS_Store')
#organize class names by alphabetical order
class_names.sort()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.jit.load(path_to_model)
model.eval()

RecursiveScriptModule(
  original_name=ResNet
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
  (relu): RecursiveScriptModule(original_name=ReLU)
  (maxpool): RecursiveScriptModule(original_name=MaxPool2d)
  (layer1): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_name=Conv2d)
      (bn2): RecursiveScriptModule(original_name=BatchNorm2d)
    )
    (1): RecursiveScriptModule(
      original_name=BasicBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (bn1): RecursiveScriptModule(original_name=BatchNorm2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_name=Conv2d)

In [22]:
evaluate(model, path_to_folder,3)



~~~~~~~~~~~~~~~~~~~~~~~EVALUATION~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
standing:
{'true_positive': 15.0, 'true_negative': 56.0, 'false_positive': 0.0, 'false_negative': 0.0}
catagory metrics: 
{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}


sitting:
{'true_positive': 28.0, 'true_negative': 43.0, 'false_positive': 0.0, 'false_negative': 0.0}
catagory metrics: 
{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}


laying:
{'true_positive': 28.0, 'true_negative': 43.0, 'false_positive': 0.0, 'false_negative': 0.0}
catagory metrics: 
{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}


########################################
average evaluation metrics: 
accuracy: 1.0
precision: 1.0
recall: 1.0
f1_score: 1.0
########################################


{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}

>>>>>>NEW MODEL SET

In [23]:
model = torch.jit.load('/Users/david/Desktop/shelter_dogs_research/Trials/trial1_pup1/Trial1_docs/trial1.1_model.pt')

In [24]:
evaluate(model, path_to_folder,3)



~~~~~~~~~~~~~~~~~~~~~~~EVALUATION~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
standing:
{'true_positive': 15.0, 'true_negative': 56.0, 'false_positive': 0.0, 'false_negative': 0.0}
catagory metrics: 
{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}


sitting:
{'true_positive': 28.0, 'true_negative': 43.0, 'false_positive': 0.0, 'false_negative': 0.0}
catagory metrics: 
{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}


laying:
{'true_positive': 28.0, 'true_negative': 43.0, 'false_positive': 0.0, 'false_negative': 0.0}
catagory metrics: 
{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}


########################################
average evaluation metrics: 
accuracy: 1.0
precision: 1.0
recall: 1.0
f1_score: 1.0
########################################


{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}

In [25]:
model = torch.jit.load('/Users/david/Desktop/shelter_dogs_research/Trials/trial1_pup1/Trial1_docs/trial1.2_model.pt')

In [26]:
evaluate(model, path_to_folder,3)



~~~~~~~~~~~~~~~~~~~~~~~EVALUATION~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
standing:
{'true_positive': 15.0, 'true_negative': 56.0, 'false_positive': 0.0, 'false_negative': 0.0}
catagory metrics: 
{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}


sitting:
{'true_positive': 28.0, 'true_negative': 43.0, 'false_positive': 0.0, 'false_negative': 0.0}
catagory metrics: 
{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}


laying:
{'true_positive': 28.0, 'true_negative': 43.0, 'false_positive': 0.0, 'false_negative': 0.0}
catagory metrics: 
{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}


########################################
average evaluation metrics: 
accuracy: 1.0
precision: 1.0
recall: 1.0
f1_score: 1.0
########################################


{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}

In [27]:
model = torch.jit.load('/Users/david/Desktop/shelter_dogs_research/Trials/trial1_pup1/Trial1_docs/trial1.2IPx_model.pt')

In [28]:
evaluate(model, '/Users/david/Desktop/shelter_dogs_research/Trials/trial1_pup1/datasets/cropped/test',3)



~~~~~~~~~~~~~~~~~~~~~~~EVALUATION~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
standing:
{'true_positive': 15.0, 'true_negative': 56.0, 'false_positive': 0.0, 'false_negative': 0.0}
catagory metrics: 
{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}


sitting:
{'true_positive': 28.0, 'true_negative': 43.0, 'false_positive': 0.0, 'false_negative': 0.0}
catagory metrics: 
{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}


laying:
{'true_positive': 28.0, 'true_negative': 43.0, 'false_positive': 0.0, 'false_negative': 0.0}
catagory metrics: 
{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}


########################################
average evaluation metrics: 
accuracy: 1.0
precision: 1.0
recall: 1.0
f1_score: 1.0
########################################


{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}